In [3]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## やりたいこと
+ 連続する数列の範囲を求める

In [5]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    col1 integer not null
);

insert into tbl
values (1,1),(2,2),(3,3),
    (4,5),(5,7),(6,8),(7,10),(8,11),(9,12);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
9 rows affected.


[]

In [18]:
%%sql
-- lagを使って、diffを求める -> 差が1でなければ足し込んでいって、階段関数にする
select group_id+1 as group_id,
    min(col1) as start_num,
    max(col1) as end_num
from (
    select *,
        -- consecutive_id相当
        sum(case when diff = 1 then 0 else 1 end) over (order by id) as group_id
    from (
        select *,
            coalesce(col1 - lag(col1,1) over (order by id),1) as diff
        from tbl
    ) as tmp1
) as tmp2
group by group_id
order by group_id

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


group_id,start_num,end_num
1,1,3
2,5,5
3,7,8
4,10,12
